## 데이터 준비

In [1]:
import pandas as pd

|    코드   |    의미                    |    창업 관점에서 적절한 점수    |
| -- | --------------------- | -------------- |
|    LH    | 신규 창업에 유리한 환경         | **4 (가장 유리)**  |
|    HL    | 기존 업체에 유리 → 창업은 다소 신중 | **3**          |
|    HH    | 진입 장벽 높음, 경쟁 심함       | **2**          |
|    LL    | 불안정, 도시재생 중 → 리스크 큼   | **1 (가장 낮음)**  |

In [2]:
df_total = pd.read_csv("./data/변수 통합_original2.csv")
df_total.head()

,자치구,인구밀도,교통 편의성,문화 인프라 지수,예술인프라지수,의료 인프라 지수,상권_변화_지표,성장가능성,학교 수
0,종로구,6292.471769,0.414197,0.320997,0.000386,0.569375,HH,2,141
1,중구,13232.228916,0.448910,1.059722,0.000152,0.883985,HH,2,171
2,용산구,10384.362140,0.254171,0.061350,0.000053,0.067072,HH,2,125
3,성동구,16930.202140,0.375717,0.005239,0.000014,0.119998,LH,4,62
4,광진구,20584.994138,0.236879,0.015377,0.000009,0.114405,LL,1,116


In [3]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   자치구        25 non-null     object 
 1   인구밀도       25 non-null     float64
 2   교통 편의성     25 non-null     float64
 3   문화 인프라 지수  24 non-null     float64
 4   예술인프라지수    25 non-null     float64
 5   의료 인프라 지수  25 non-null     float64
 6   상권_변화_지표   25 non-null     object 
 7   성장가능성      25 non-null     int64  
 8   학교 수       25 non-null     int64  
dtypes: float64(5), int64(2), object(2)
memory usage: 1.9+ KB


In [4]:
df = df_total.copy()

### 

### 회귀분석

In [5]:
from statsmodels.formula.api import ols, glm

# 공백이 있는 컬럼명 처리
df = df.rename(columns={
    '교통 편의성': '교통_편의성',
    '예술인프라지수': '예술인프라지수',
    '의료 인프라 지수': '의료_인프라_지수',
    '인구밀도': '인구밀도',
    '성장가능성': '성장가능성',
    '학교 수': '학교_수'
})

# 회귀 분석
Rformula = '성장가능성 ~ 인구밀도 + 교통_편의성 + 예술인프라지수 + 의료_인프라_지수 + 학교_수'
regression_result = ols(Rformula, data=df).fit()
regression_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  성장가능성   R-squared:                       0.336
Model:                            OLS   Adj. R-squared:                  0.161
Method:                 Least Squares   F-statistic:                     1.922
Date:                Tue, 17 Jun 2025   Prob (F-statistic):              0.138
Time:                        03:09:25   Log-Likelihood:                -30.847
No. Observations:                  25   AIC:                             73.69
Df Residuals:                      19   BIC:                             81.01
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.0790      1.203      0.897      0.381      -1.438       3.596
인구밀도        4.053e-05   5.24e-05      0.773      0.449   -6.92e-05       0.000
교통_편의성         3.5989      1.390      2.588      0.018       0.689       6.509
예술인프라지수     5274.9554   3248.700      1.624      0.121   -1524.651    1.21e+04
의료_인프라_지수     -1.7468      1.026     -1.702      0.105      -3.895       0.402
학교_수          -0.0086      0.004     -2.291      0.034      -0.017      -0.001
==============================================================================
Omnibus:                        2.155   Durbin-Watson:                   2.680
Prob(Omnibus):                  0.340   Jarque-Bera (JB):                1.425
Skew:                           0.585   Prob(JB):                        0.491
Kurtosis:                       2.971   Cond. No.                     2.96e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.96e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

<b>결과:</b>  
- R-squared가 1에 가까울수록 회귀모델이 데이터에 잘 들어맞는 것이지만  
  위 모델은 0.336로 **데이터에 맞지 않음을 확인할 수 있다.**
  
  다만 **학교 변수 추가** 후 0.152에서 **0.336으로 증가**했음 또한 확인할 수 있다.
  
- 또한 학교_수 변수의 P-value가 0.05보다 작으므로  
  **모든 변수가 통계적으로 유의함**을 알 수 있다.
   
=> 순서혈 로지스틱 회귀 분석 수행

📊 학교 수 변수 추가 전후 회귀 성능 및 유의성 비교표
| 항목               | 학교 수 **포함 전** | 학교 수 **포함 후** | 변화 내용                        |
| ---------------- | ------------- | ------------- | ---------------------------- |
| 결정계수 (R-squared) | 0.152         | 0.336         | **+0.184** 상승                |
| 학교 수 변수의 p-value | -             | **0.034**     | 유의 수준 0.05 이하 → **통계적으로 유의** |
| 모델의 설명력          | 낮음            | **개선됨**       | 모델의 설명력이 크게 향상됨              |
| 학교 수 변수의 채택 필요성  | 불명확           | **도입이 타당**    | 예측력 향상 및 통계적으로 유의            |


### 

### 로지스틱 회귀

In [8]:
from sklearn.preprocessing import StandardScaler
from mord import LogisticAT

# 🔹 1. X, y 정의
X_cols = ['인구밀도', '교통_편의성', '예술인프라지수', '의료_인프라_지수', '학교_수']
X = df[X_cols]
y = df['성장가능성']

# 🔹 2. 표준화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 🔹 3. 순서형 로지스틱 회귀 모델 훈련
model = LogisticAT()  # LogisticAT = proportional odds model
model.fit(X_scaled, y)

# 🔹 4. 예측 (선택)
y_pred = model.predict(X_scaled)

# 🔹 5. 결과 확인
from sklearn.metrics import classification_report, confusion_matrix
print("분류 리포트:\n", classification_report(y, y_pred))
print("혼동 행렬:\n", confusion_matrix(y, y_pred))

분류 리포트:
               precision    recall  f1-score   support

           1       0.77      0.77      0.77        13
           2       0.36      0.57      0.44         7
           3       1.00      0.50      0.67         2
           4       0.00      0.00      0.00         3

    accuracy                           0.60        25
   macro avg       0.53      0.46      0.47        25
weighted avg       0.58      0.60      0.58        25

혼동 행렬:
 [[10  3  0  0]
 [ 3  4  0  0]
 [ 0  1  1  0]
 [ 0  3  0  0]]


C:\venvs\py37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\venvs\py37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\venvs\py37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<b>정확도(accuracy) : 0.60</b>  
총 25개 샘플 중 15개를 정확히 예측함.  
학교 변수 추가 전 0.44인 것을 감안하면 유의미한 개선이 발생했음을 알 수 있다.  
클래스 수(4개) 대비 baseline이 약 25%인 점을 고려하면 단순 추정보다 성능은 높지만, **일부 클래스에서 심각한 불균형과 오분류가 발생**하고 있어 모델 개선이 필요함.

  
학교 변수 추가 전  
| 성장가능성 | Precision | Recall | F1-score | Support | 해석                  |
| ----- | --------- | ------ | -------- | ------- | ------------------- |
| 1     | 0.57      | 0.62   | 0.59     | 13      | 수가 많고 비교적 안정적으로 예측  |
| 2     | 0.20      | 0.29   | 0.24     | 7       | 예측 정확도 낮고 혼동 많음     |
| 3     | 1.00      | 0.50   | 0.67     | 2       | 예측 대상 적어 신뢰도 낮음     |
| 4     | 0.00      | 0.00   | 0.00     | 3       | 전혀 예측 못함 (Recall 0) |

학교 변수 추가 후  
| 성장가능성 | Precision | Recall | F1-score | Support | 해석                                  |
| ----- | --------- | ------ | -------- | ------- | ----------------------------------- |
| 1     | 0.77      | 0.77   | 0.77     | 13      | 가장 데이터 수가 많고, 안정적으로 예측됨             |
| 2     | 0.36      | 0.57   | 0.44     | 7       | 과반수 예측 성공했지만 precision 낮음, 일부 혼동 존재 |
| 3     | 1.00      | 0.50   | 0.67     | 2       | 정확히 맞춘 경우는 있지만 예측 대상이 적어 신뢰도 낮음     |
| 4     | 0.00      | 0.00   | 0.00     | 3       | **전혀 예측되지 않음**, recall 0 → 모델 개선 필요 |

### 